# Loop flow example - Refer to Nelleka's work
This is a test case with particles released in a simple looping flow field 
The background flow velocity is given by 

$u=-sin(b*x)*sin(b*y),\ b=2pi/L$
$v=cos(a*x)*sin(a*y), a=pi/H$

The domain is $H \in [0,50], L \in [0,100]$

In [1]:
# Load required libraries
using Pkg
Pkg.instantiate()
Pkg.activate("..")
using Particles
using Plots
using Random

randpool = MersenneTwister(0) #Generate same random numbers every time

nothing

  Activating project at `d:\Projects\particles.jl`


In [6]:
# Define the flow field
H=50
L=2*H # divergence-free condition
a=pi/H
b=2*pi/L

# sign difference from stream function to velocity

# forcing currents stream (used only for plotting here) , u and v
# flow in x direction (for now has to be called u)
function u(x, y, z, t)
    -sin(b * x) * cos(b * y)
end

# flow in y direction (for now has to be called v)
function v(x, y, z, t)
    cos(a* x) * sin(a * y)
end

nothing

In [7]:
function init_setup(d::AbstractDict)
    # Load default settings and adjust
    # collected configuration is in Dict d
    d["rng"] = randpool
    # settings for this experiment
    n = 15 # number of particles
    d["nparticles"] = n
    d["nparticles_bound"] = n
    # all variables for one particle are collected in a vector
    variables = ["x", "y", "z", "age"]
    d["variables"] = variables
    # initial position of the particles
    m = length(variables)
    p = zeros(m, n)
    p[1, :] += 50 .+ 1 * randn(randpool, n, 1) # particles are spawned roughly at the center of the graph
    p[2, :] += 25 .+ 1 * randn(randpool, n, 1)
    d["particles"] = p # initial values
    display(d["particles"])

    t = zeros(2,2)
    t[1,:] .-= 1
    t[2,:] .+= 2
    d["temp"] = t
    display(d["temp"])

    # simulation time
    d["dt"] = 0.05     #time-step
    d["tstart"] = 0.0 
    d["tend"] = 150
    # write to netcdf
    d["write_maps_times"] = collect(d["tstart"]:d["dt"]:d["tend"])
    d["write_maps"] = false #do not write to netcdf
    d["write_maps_filename"] = "output_loop.nc"
    # write plots to file
    d["plot_maps_times"] = collect(d["tstart"]:d["dt"]:d["tend"])
    d["plot_maps"] = false # do not make png figures
    d["plot_maps_size"]=(600,300)

    # keep some output in memory
    d["keep_particles"] = true #keep results in memory (bad idea for a large run)
    d["keep_particle_times"] = collect(d["tstart"]:20*d["dt"]:d["tend"])

    #d["particles"] = zeros(length(variables), d["nparticles_bound"])
    d["is_particles_active"] = falses(d["nparticles_bound"],)
    d["is_keep_these_particles"] = trues(d["nparticles_bound"],)
    
    # the following options are to be added in the default_userdata() function
    # if to apply the algorithm to prevent the algorithm from crossing the boundary or not
    d["is_cross_bc"] = false # if added, the algorithm will terminate once the particle is out of the boundary
    d["is_apply_cross_bc_test"] = false # if applied, only one particle is used and all virtual trajectories are also recorded
    if d["is_apply_cross_bc_test"]
        d["the_virtual_trajectory"] = d["particles"]
    end
    # to be added according to the actual problem
    d["isactive_particles"] = trues(d["nparticles_bound"],) # check if particles move out of the is_particle_in_domain
    d["recursion_depth"] = 10
    
    # to be added according to the actual problem
    function is_particle_in_domain(x)
        return true, false # true-->is_next_inside_domain, false-->is_apply_reflection
    end
    d["is_particle_in_domain"] = is_particle_in_domain
    
    # magnitude of the diffusion profile
    # set figname
    #figbkg="single_particle_test_result\\bkg_K=$(K_MAG).png"
    #figname="single_particle_test_result\\selected_results\\sprt_K=$(K_MAG).png"
    
    # function to implement void IC and forcing conditions: fixed number of particles
    d["first_active_index"] = 1
    d["first_inactive_index"] = d["nparticles_bound"] + 1
    

    # code for 
    function initialize_particles(d,p)
        return  p
    end
    d["initialize_particles"] = initialize_particles
    
    function release_particles(d,p,t)
        return p
    end
    d["release_particles"] = release_particles

    return d #show configuration
end

init_setup (generic function with 1 method)

In [19]:
#plot flow field
#grid for plotting
x = 0:5:100
y = 0:5:50
xs=kron(x,ones(length(y)))
ys=kron(ones(length(x)),y)
Plots.default(:size, (600,300))
scale = 5
u_plot = zeros(length(x)*length(y))
v_plot = zeros(length(x)*length(y))
for i = 1:length(x)
    for j = 1:length(y)
        u_plot[(i-1)*length(y)+j] = u(x[i], y[j], 0.0, 0.0)
        v_plot[(i-1)*length(y)+j] = v(x[i], y[j], 0.0, 0.0)
    end
end
quiver(xs,ys,quiver=(scale*u_plot,scale*v_plot))
#savefig(".\\vortex\\eps1\\vortex.png")

"d:\\Projects\\particles.jl\\example_notebooks\\vortex\\eps1\\vortex.png"

In [9]:
# Here is the equation that we want to solve for the particles

"""
   !f(ds,s,t,i,d)

Dynamic model, computes time derivative ds of s at current time t
for particle i and possibly using data/functions from d of type userdata.
"""
function f!(∂s, s, t, i, d)
   x, y, z, age = s
   # dx/dt=u
    ∂s.x = u(x, y, z, t)
   # dy/dt=v
    ∂s.y = v(x, y, z, t)
   # dz/dt=0
    ∂s.z = 0.0
   # age=(t-t0)
    ∂s.t = 1.0
end
#d["f"] = f!
# Here is the equation that we want to solve for the particles

"""
   g!(ds,s,t,i,d)

Dynamic model, computes time derivative ds of s at current time t
for particle i and possibly using data/functions from d of type userdata.
"""
function g!(∂s, s, t, i, d)
   x, y, z, age = s
   # dx/dt=u
    ∂s.x = 0.0
   # dy/dt=v
    ∂s.y = 0.0
   # dz/dt=0
    ∂s.z = 0.0
   # age=(t-t0)
    ∂s.t = 0.0
end

g!

In [10]:
###### Velocity function for the particles ######
# some (intermediate) parameters can be precomputed
# for the time being all parameters are computed again
# construct a struct class named Params to pass keyword arguments locally
function f1!(ds, s, t, i, d; bc_treat="original", L=L, H=H)
    x, y, z, age = s
    z = 0.0
    up = 0
    vp = 0
    dt = d["dt"]
    uw = u(x, y, z, t)
    vw = v(x, y, z, t)
 
    # add background flow velocity
    up += uw
    vp += vw
 
    # specify gaussian diffusion parameters
    epsx = 1
    epsz = 1e-5  
 
    # add bc treament to fix particles diffusion problem
    if bc_treat=="original"
       up += randn()*sqrt(2*epsx*dt)/dt
       vp += randn()*sqrt(2*epsz*dt)/dt
    elseif bc_treat == "reflection"
       # assume a wall at the bc some particles will bounce back without losing kinetic energy
       up += randn()*sqrt(2*epsx*dt)/dt
       vp += randn()*sqrt(2*epsz*dt)/dt
       xp = x + up*dt # xp and yp are virtual particle location
       yp = y + vp*dt
       # reflection in horizontal direction
       xsurplus = 0
       if xp < 0
          xsurplus = xp - 0
       elseif xp > L
          xsurplus = xp - L
       end
       up -= 2 * xsurplus / dt
       # reflection in vertical direction
       ysurplus = 0
       if yp < 0
          ysurplus = yp - 0
       elseif yp > H
          ysurplus = yp - H
       end
       vp -= 2 * ysurplus / dt
    elseif bc_treat == "suppression"
       # supress diffusion near the boundary
       # a criterion of 3-sigma is applied
       bcx_THICKNESS = sqrt(2*epsx) * (3*sqrt(dt))
       bcy_THICKNESS = sqrt(2*epsz) * (3*sqrt(dt))
       if abs(x)>bcx_THICKNESS && abs(x-L)>bcx_THICKNESS
          up += randn()*sqrt(2*epsx*dt)/dt
       end
       if abs(y)>bcy_THICKNESS && abs(y-H)>bcy_THICKNESS
          vp += randn()*sqrt(2*epsz*dt)/dt
       end
    end
    # record the particle velocity
    ds.x = up
    ds.y = vp
    ds.z = 0.0
    ds.t = 1.0
 
    if d["time_direction"] == :backwards
       up *= -1
       vp *= -1
    end
 end
 #bc_treat = "original"
 #function fhelp!(ds, s, t, i, d; bc_treat=bc_treat) f1!(ds, s, t, i, d; bc_treat=bc_treat) end
 #d["f"]= fhelp!

f1! (generic function with 1 method)

In [11]:
# use streamfunction as background for plotting

function plot_background(d)
    x = 0:5:100
    y = 0:5:50
    xs=kron(x,ones(length(y)))
    ys=kron(ones(length(x)),y)
    Plots.default(:size, d["plot_maps_size"])
    scale = 5
    u_plot = zeros(length(x)*length(y))
    v_plot = zeros(length(x)*length(y))
    for i = 1:length(x)
        for j = 1:length(y)
            u_plot[(i-1)*length(y)+j] = u(x[i], y[j], 0.0, 0.0)
            v_plot[(i-1)*length(y)+j] = v(x[i], y[j], 0.0, 0.0)
        end
    end
    f = quiver(xs,ys,quiver=(scale*u_plot,scale*v_plot))
    return(f)
 end
# d["plot_maps_background"] = plot_background
 

plot_background (generic function with 1 method)

In [12]:
# screenshot of particles
function plot_screenshot(d; dirname="", fnametag="")
  t_plot=d["keep_particle_times"];
  p=d["all_particles"]
  for i in 1:length(t_plot)
      fig = d["plot_maps_background"](d)
      fname = dirname*"\\screenshot_time"*"$(t_plot[i])"*".png"
      d["plot_maps_func"](fig, d, p[i])
      savefig(fig, fname)
  end
  nothing
end

function plot_trajectories(d; dirname="", fnametag="")
    # plot particles' trajectories
    t_plot = d["keep_particle_times"];
    p = d["all_particles"]
    n = d["nparticles"]
    p_x = zeros(length(t_plot), n)
    p_y = zeros(length(t_plot), n)
    for i in 1:length(t_plot)
        p_x[i, :] = p[i][1,:]
        p_y[i, :] = p[i][2,:]
    end

    # prepare the data for plotting
    # note that the data points at the first and the end are used once
    indices = reduce(vcat, [[j for i in 1:2] for j in 1:length(t_plot)]) #reduce a vector of vectors to a vector
    indices = indices[begin+1:end-1]
    fig = d["plot_maps_background"](d)
    fname = dirname*"\\trajectories_$(fnametag).png"

    for i in 1:n
        plot!(fig, p_x[indices, i], p_y[indices, i], 
               legend=egend =:outerright, 
               size = (1400, 800),
               linewidth = 2.5,
               markershape = :circle,
               markersize = 1.5,
               markerstrokecolor=:auto,
               label = "particle$(i)")
    end

    savefig(fig, fname)
end

plot_trajectories (generic function with 1 method)

In [13]:
# compute the initial position of particles
function get_p0()
    # Load default settings and adjust
    # collected configuration is in Dict d
    d = default_userdata() # start with some defaults
    # settings for this experiment
    n = 15 # number of particles
    d["nparticles"] = n
    # all variables for one particle are collected in a vector
    variables = ["x", "y", "z", "age"]
    d["variables"] = variables
    # initial position of the particles
    m = length(variables)
    p = zeros(m, n)
    p[1, :] = 50 .+ 1 * randn(randpool, n, 1) # particles are spawned roughly at the center of the graph
    p[2, :] = 25 .+ 1 * randn(randpool, n, 1)
    d["particles"] = p # initial values
    p;
end 
p0 = get_p0()

4×15 Matrix{Float64}:
 51.423   50.4084  50.5886  49.7037  …  49.6781  50.2463  49.8212  48.5221
 24.8146  26.2697  24.8375  24.9328     25.512   24.2413  24.232   24.9697
  0.0      0.0      0.0      0.0         0.0      0.0      0.0      0.0
  0.0      0.0      0.0      0.0         0.0      0.0      0.0      0.0

In [14]:
print(p0)

[51.423048580664314 50.40838681696171 50.58862138060246 49.703722302890924 50.69111073411025 50.50687363404227 49.943070123901045 48.22897635462786 51.590622520307406 51.39706144739974 50.48155603952425 49.67805684710403 50.24634263300826 49.82121126768698 48.52211927507813; 24.81457581826701 26.269723691035253 24.837496135667518 24.93281329471806 25.407256743931118 25.57728198659179 25.891315091076542 22.705509656726495 23.826974247731158 24.120084500834714 24.505957340195692 25.511959302312157 24.241299016905078 24.23204016049694 24.96969682367943; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]

In [15]:
dirname = "vortex\\eps$(1)"
if ~isdir(dirname)
    mkdir(dirname)
end
dirname = joinpath(@__DIR__, dirname)

"d:\\Projects\\particles.jl\\example_notebooks\\vortex\\eps1"

In [16]:
open("stdout.txt","w") do io
redirect_stdout(io) do
    for cycle_number in 1:2
        for bc_treat in ["original", "reflection", "suppression"]
            d = default_userdata() # start with some defaults
            d = init_setup(d)
            #d["particles"] = p0
            function fhelp!(ds, s, t, i, d; bc_treat=bc_treat) f1!(ds, s, t, i, d; bc_treat=bc_treat) end
            d["f"]= fhelp!
            d["g"]= g!
            d["plot_maps_background"] = plot_background
            @time run_simulation(d)
            plot_trajectories(d; dirname=dirname, fnametag="$(bc_treat)_cycle$cycle_number")
        end
    end
end
end

4×15 Matrix{Float64}:
 49.5174  48.8425  49.7858  48.6281  …  50.8162  50.1525  49.6699  51.4443
 24.5572  26.0969  24.7489  25.2923     25.5567  24.4311  24.5231  25.1746
  0.0      0.0      0.0      0.0         0.0      0.0      0.0      0.0
  0.0      0.0      0.0      0.0         0.0      0.0      0.0      0.0

2×2 Matrix{Float64}:
 -1.0  -1.0
  2.0   2.0

4×15 Matrix{Float64}:
 48.8662  50.3741  51.3054  49.1607  …  49.2601  50.1492  51.7469  48.5526
 24.7391  26.1491  23.3555  24.5859     25.1731  26.8887  24.1203  25.8821
  0.0      0.0      0.0      0.0         0.0      0.0      0.0      0.0
  0.0      0.0      0.0      0.0         0.0      0.0      0.0      0.0

2×2 Matrix{Float64}:
 -1.0  -1.0
  2.0   2.0

4×15 Matrix{Float64}:
 50.3779  48.9999  49.4358  49.0184  …  48.83    52.2451  50.2471  49.113
 23.3432  22.7802  26.6063  25.9533     24.5514  25.4383  24.694   24.0968
  0.0      0.0      0.0      0.0         0.0      0.0      0.0      0.0
  0.0      0.0      0.0      0.0         0.0      0.0      0.0      0.0

...

2×2 Matrix{Float64}:
 -1.0  -1.0
  2.0   2.0

4×15 Matrix{Float64}:
 49.9927  49.8323  49.6211  49.8126  …  51.2676  47.9762  49.7277  50.6587
 23.8034  25.7244  23.6797  23.6761     25.1359  25.9838  25.6053  24.4204
  0.0      0.0      0.0      0.0         0.0      0.0      0.0      0.0
  0.0      0.0      0.0      0.0         0.0      0.0      0.0      0.0

2×2 Matrix{Float64}:
 -1.0  -1.0
  2.0   2.0

4×15 Matrix{Float64}:
 49.1993  49.4438  49.5444  51.0773  …  50.1378  51.6397  50.5625  49.5101
 25.8971  25.3904  25.0012  25.1999     24.2487  23.5232  24.6789  23.925
  0.0      0.0      0.0      0.0         0.0      0.0      0.0      0.0
  0.0      0.0      0.0      0.0         0.0      0.0      0.0      0.0

2×2 Matrix{Float64}:
 -1.0  -1.0
  2.0   2.0

In [17]:
# t_plot=d["keep_particle_times"];
# p=d["all_particles"]

# display(size(p))
# display(typeof(p))
# display(size(p[1]))
# display(size(p[:][:,1]))
# display(size(p[:][1,1]))
# display(typeof(p[:][1,1]))

# display(size(p[1][1,:]))
# display(typeof(p[1][1,:]))

# # obtain the location data
# display(d["variables"])
# display(length(t_plot))
# display("Particle number")
# display(n)